# Context Engineering 연구 노트북

DeepAgents 라이브러리에서 사용되는 5가지 Context Engineering 전략을 분석하고 실험합니다.

## Context Engineering 5가지 핵심 전략

| 전략 | 설명 | DeepAgents 구현 |
|------|------|----------------|
| **1. Offloading** | 대용량 결과를 파일로 축출 | FilesystemMiddleware |
| **2. Reduction** | Compaction + Summarization | SummarizationMiddleware |
| **3. Retrieval** | grep/glob 기반 검색 | FilesystemMiddleware |
| **4. Isolation** | SubAgent로 컨텍스트 격리 | SubAgentMiddleware |
| **5. Caching** | Prompt Caching | AnthropicPromptCachingMiddleware |

## 아키텍처 개요

```
┌─────────────────────────────────────────────────────────────────┐
│                     Context Engineering                          │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│   ┌────────────┐    ┌────────────┐    ┌────────────┐            │
│   │ Offloading │    │ Reduction  │    │  Caching   │            │
│   │ (20k 토큰) │    │ (85% 임계) │    │ (Anthropic)│            │
│   └─────┬──────┘    └─────┬──────┘    └─────┬──────┘            │
│         │                 │                 │                    │
│         ▼                 ▼                 ▼                    │
│   ┌─────────────────────────────────────────────────────┐       │
│   │              Middleware Stack                       │       │
│   └─────────────────────────────────────────────────────┘       │
│                          │                                       │
│         ┌────────────────┼────────────────┐                     │
│         ▼                ▼                ▼                     │
│   ┌────────────┐  ┌────────────┐  ┌────────────┐               │
│   │ Retrieval  │  │ Isolation  │  │  Backend   │               │
│   │(grep/glob) │  │ (SubAgent) │  │ (FileSystem│               │
│   └────────────┘  └────────────┘  └────────────┘               │
│                                                                  │
└─────────────────────────────────────────────────────────────────┘
```

In [ ]:
import sys
from pathlib import Path

from dotenv import load_dotenv

load_dotenv(".env", override=True)

PROJECT_ROOT = Path.cwd()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

---

## 전략 1: Context Offloading

대용량 도구 결과를 파일시스템으로 축출하여 컨텍스트 윈도우 오버플로우를 방지합니다.

### 핵심 원리
- 도구 결과가 `tool_token_limit_before_evict` (기본 20,000 토큰) 초과 시 자동 축출
- `/large_tool_results/{tool_call_id}` 경로에 저장
- 처음 10줄 미리보기 제공
- 에이전트가 `read_file`로 필요할 때 로드

In [ ]:
from context_engineering_research_agent.context_strategies.offloading import (
    ContextOffloadingStrategy,
    OffloadingConfig,
)

config = OffloadingConfig(
    token_limit_before_evict=20000,
    eviction_path_prefix="/large_tool_results",
    preview_lines=10,
)

print(f"토큰 임계값: {config.token_limit_before_evict:,}")
print(f"축출 경로: {config.eviction_path_prefix}")
print(f"미리보기 줄 수: {config.preview_lines}")

In [ ]:
strategy = ContextOffloadingStrategy(config=config)

small_content = "짧은 텍스트" * 100
large_content = "대용량 텍스트" * 30000

print(f"짧은 콘텐츠: {len(small_content)} 자 → 축출 대상: {strategy._should_offload(small_content)}")
print(f"대용량 콘텐츠: {len(large_content):,} 자 → 축출 대상: {strategy._should_offload(large_content)}")

---

## 전략 2: Context Reduction

컨텍스트 윈도우 사용량이 임계값을 초과할 때 자동으로 대화 내용을 압축합니다.

### 두 가지 기법

| 기법 | 설명 | 비용 |
|------|------|------|
| **Compaction** | 오래된 도구 호출/결과 제거 | 무료 |
| **Summarization** | LLM이 대화 요약 | API 비용 발생 |

우선순위: Compaction → Summarization

In [ ]:
from context_engineering_research_agent.context_strategies.reduction import (
    ContextReductionStrategy,
    ReductionConfig,
)

config = ReductionConfig(
    context_threshold=0.85,
    model_context_window=200000,
    compaction_age_threshold=10,
    min_messages_to_keep=5,
)

print(f"임계값: {config.context_threshold * 100}%")
print(f"컨텍스트 윈도우: {config.model_context_window:,} 토큰")
print(f"Compaction 대상 나이: {config.compaction_age_threshold} 메시지")
print(f"최소 유지 메시지: {config.min_messages_to_keep}")

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage

strategy = ContextReductionStrategy(config=config)

messages = [
    HumanMessage(content="안녕하세요" * 1000),
    AIMessage(content="안녕하세요" * 1000),
] * 20

usage_ratio = strategy._get_context_usage_ratio(messages)
print(f"컨텍스트 사용률: {usage_ratio * 100:.1f}%")
print(f"축소 필요: {strategy._should_reduce(messages)}")

---

## 전략 3: Context Retrieval

grep/glob 기반의 단순하고 빠른 검색으로 필요한 정보만 선택적으로 로드합니다.

### 벡터 검색을 사용하지 않는 이유

| 특성 | 직접 검색 | 벡터 검색 |
|------|----------|----------|
| 결정성 | ✅ 정확한 매칭 | ❌ 확률적 |
| 인프라 | ✅ 불필요 | ❌ 벡터 DB 필요 |
| 속도 | ✅ 빠름 | ❌ 인덱싱 오버헤드 |
| 디버깅 | ✅ 예측 가능 | ❌ 블랙박스 |

In [ ]:
from context_engineering_research_agent.context_strategies.retrieval import (
    ContextRetrievalStrategy,
    RetrievalConfig,
)

config = RetrievalConfig(
    default_read_limit=500,
    max_grep_results=100,
    max_glob_results=100,
    truncate_line_length=2000,
)

print(f"기본 읽기 제한: {config.default_read_limit} 줄")
print(f"grep 최대 결과: {config.max_grep_results}")
print(f"glob 최대 결과: {config.max_glob_results}")
print(f"줄 길이 제한: {config.truncate_line_length} 자")

---

## 전략 4: Context Isolation

SubAgent를 통해 독립된 컨텍스트 윈도우에서 작업을 수행합니다.

### 장점
- 메인 에이전트 컨텍스트 오염 방지
- 복잡한 작업의 격리 처리
- 병렬 처리 가능

### SubAgent 유형

| 유형 | 구조 | 특징 |
|------|------|------|
| Simple | `{name, system_prompt, tools}` | 단일 응답 |
| Compiled | `{name, runnable}` | 자체 DeepAgent, 다중 턴 |

In [ ]:
from context_engineering_research_agent.context_strategies.isolation import (
    ContextIsolationStrategy,
    IsolationConfig,
)

config = IsolationConfig(
    default_model="gpt-4.1",
    include_general_purpose_agent=True,
    excluded_state_keys=("messages", "todos", "structured_response"),
)

print(f"기본 모델: {config.default_model}")
print(f"범용 에이전트 포함: {config.include_general_purpose_agent}")
print(f"제외 상태 키: {config.excluded_state_keys}")

---

## 전략 5: Context Caching

Anthropic Prompt Caching을 활용하여 시스템 프롬프트와 반복 컨텍스트를 캐싱합니다.

### 이점
- API 호출 비용 절감
- 응답 속도 향상
- 동일 세션 내 반복 호출 최적화

### 캐싱 조건
- 최소 1,024 토큰 이상
- `cache_control: {"type": "ephemeral"}` 마커 추가

In [ ]:
from context_engineering_research_agent.context_strategies.caching import (
    ContextCachingStrategy,
    CachingConfig,
)

config = CachingConfig(
    min_cacheable_tokens=1024,
    cache_control_type="ephemeral",
    enable_for_system_prompt=True,
    enable_for_tools=True,
)

print(f"최소 캐싱 토큰: {config.min_cacheable_tokens:,}")
print(f"캐시 컨트롤 타입: {config.cache_control_type}")
print(f"시스템 프롬프트 캐싱: {config.enable_for_system_prompt}")
print(f"도구 캐싱: {config.enable_for_tools}")

In [ ]:
strategy = ContextCachingStrategy(config=config)

short_content = "짧은 시스템 프롬프트"
long_content = "긴 시스템 프롬프트 " * 500

print(f"짧은 콘텐츠: {len(short_content)} 자 → 캐싱 대상: {strategy._should_cache(short_content)}")
print(f"긴 콘텐츠: {len(long_content):,} 자 → 캐싱 대상: {strategy._should_cache(long_content)}")

---

## 통합 에이전트 실행

5가지 전략이 모두 적용된 에이전트를 실행합니다.

In [ ]:
from context_engineering_research_agent import create_context_aware_agent

agent = create_context_aware_agent(
    model_name="gpt-4.1",
    enable_offloading=True,
    enable_reduction=True,
    enable_caching=True,
    offloading_token_limit=20000,
    reduction_threshold=0.85,
)

print(f"에이전트 타입: {type(agent).__name__}")

---

## 전략 활성화/비활성화 비교 실험

각 전략을 활성화/비활성화했을 때의 차이점을 실험합니다.

### 실험 설계

| 실험 | Offloading | Reduction | Caching | 목적 |
|------|------------|-----------|---------|------|
| 1. 기본 | ❌ | ❌ | ❌ | 베이스라인 |
| 2. Offloading만 | ✅ | ❌ | ❌ | 대용량 결과 축출 효과 |
| 3. Reduction만 | ❌ | ✅ | ❌ | 컨텍스트 압축 효과 |
| 4. 모두 활성화 | ✅ | ✅ | ✅ | 전체 효과 |

In [ ]:
from context_engineering_research_agent.context_strategies.offloading import (
    ContextOffloadingStrategy, OffloadingConfig
)
from context_engineering_research_agent.context_strategies.reduction import (
    ContextReductionStrategy, ReductionConfig
)
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage

print("=" * 60)
print("전략 비교를 위한 테스트 데이터 생성")
print("=" * 60)

### 실험 1: Offloading 전략 효과

대용량 도구 결과가 있을 때 Offloading 활성화/비활성화 비교

In [ ]:
small_result = "검색 결과: 항목 1, 항목 2, 항목 3"
large_result = "\n".join([f"검색 결과 {i}: " + "상세 내용 " * 100 for i in range(500)])

print(f"작은 결과 크기: {len(small_result):,} 자")
print(f"대용량 결과 크기: {len(large_result):,} 자")
print()

offloading_disabled = ContextOffloadingStrategy(
    config=OffloadingConfig(token_limit_before_evict=999999999)
)
offloading_enabled = ContextOffloadingStrategy(
    config=OffloadingConfig(token_limit_before_evict=20000)
)

print("[Offloading 비활성화 시]")
print(f"  작은 결과 축출: {offloading_disabled._should_offload(small_result)}")
print(f"  대용량 결과 축출: {offloading_disabled._should_offload(large_result)}")
print(f"  → 대용량 결과가 컨텍스트에 그대로 포함됨")
print()

print("[Offloading 활성화 시]")
print(f"  작은 결과 축출: {offloading_enabled._should_offload(small_result)}")
print(f"  대용량 결과 축출: {offloading_enabled._should_offload(large_result)}")
print(f"  → 대용량 결과는 파일로 저장, 미리보기만 컨텍스트에 포함")
print()

preview = offloading_enabled._create_preview(large_result)
print(f"미리보기 크기: {len(preview):,} 자 (원본의 {len(preview)/len(large_result)*100:.1f}%)")

### 실험 2: Reduction 전략 효과

긴 대화에서 Compaction 적용 전/후 비교

In [ ]:
messages_with_tools = []
for i in range(30):
    messages_with_tools.append(HumanMessage(content=f"질문 {i}: " + "내용 " * 50))
    ai_msg = AIMessage(
        content=f"답변 {i}: " + "응답 " * 50,
        tool_calls=[{'id': f'call_{i}', 'name': 'search', 'args': {'q': 'test'}}] if i < 25 else []
    )
    messages_with_tools.append(ai_msg)
    if i < 25:
        messages_with_tools.append(ToolMessage(content=f"도구 결과 {i}: " + "결과 " * 30, tool_call_id=f'call_{i}'))

reduction = ContextReductionStrategy(
    config=ReductionConfig(compaction_age_threshold=10)
)

original_tokens = reduction._estimate_tokens(messages_with_tools)
print(f"[Reduction 비활성화 시]")
print(f"  메시지 수: {len(messages_with_tools)}")
print(f"  추정 토큰: {original_tokens:,}")
print(f"  → 모든 도구 호출/결과가 컨텍스트에 유지됨")
print()

compacted, result = reduction.apply_compaction(messages_with_tools)
compacted_tokens = reduction._estimate_tokens(compacted)

print(f"[Reduction 활성화 시 - Compaction]")
print(f"  메시지 수: {len(messages_with_tools)} → {len(compacted)}")
print(f"  추정 토큰: {original_tokens:,} → {compacted_tokens:,}")
print(f"  절약된 토큰: {result.estimated_tokens_saved:,} ({result.estimated_tokens_saved/original_tokens*100:.1f}%)")
print(f"  → 오래된 도구 호출/결과가 제거되어 컨텍스트 효율화")

### 실험 3: 전략 조합 효과 시뮬레이션

모든 전략을 함께 적용했을 때의 시너지 효과

In [ ]:
print("=" * 60)
print("시나리오: 복잡한 연구 작업 수행")
print("=" * 60)
print()

scenario = {
    "대화 턴 수": 50,
    "도구 호출 수": 40,
    "대용량 결과 수": 5,
    "평균 결과 크기": "100k 자",
}

print("[시나리오 설정]")
for k, v in scenario.items():
    print(f"  {k}: {v}")
print()

baseline_context = 50 * 500 + 40 * 300 + 5 * 100000
print("[모든 전략 비활성화 시]")
print(f"  예상 컨텍스트 크기: {baseline_context:,} 자 (~{baseline_context//4:,} 토큰)")
print(f"  문제: 컨텍스트 윈도우 초과 가능성 높음")
print()

with_offloading = 50 * 500 + 40 * 300 + 5 * 1000
print("[Offloading만 활성화 시]")
print(f"  예상 컨텍스트 크기: {with_offloading:,} 자 (~{with_offloading//4:,} 토큰)")
print(f"  절약: {(baseline_context - with_offloading):,} 자 ({(baseline_context - with_offloading)/baseline_context*100:.1f}%)")
print()

with_reduction = with_offloading * 0.6
print("[Offloading + Reduction 활성화 시]")
print(f"  예상 컨텍스트 크기: {int(with_reduction):,} 자 (~{int(with_reduction)//4:,} 토큰)")
print(f"  총 절약: {int(baseline_context - with_reduction):,} 자 ({(baseline_context - with_reduction)/baseline_context*100:.1f}%)")
print()

print("[+ Caching 활성화 시 추가 효과]")
print(f"  시스템 프롬프트 캐싱으로 반복 호출 비용 90% 절감")
print(f"  응답 속도 향상")

### 실험 4: 실제 에이전트 실행 비교

실제 에이전트를 다른 설정으로 생성하여 비교합니다.

In [ ]:
from context_engineering_research_agent import create_context_aware_agent

print("에이전트 생성 비교")
print("=" * 60)

configs = [
    {"name": "기본 (모두 비활성화)", "offloading": False, "reduction": False, "caching": False},
    {"name": "Offloading만", "offloading": True, "reduction": False, "caching": False},
    {"name": "Reduction만", "offloading": False, "reduction": True, "caching": False},
    {"name": "모두 활성화", "offloading": True, "reduction": True, "caching": True},
]

for cfg in configs:
    agent = create_context_aware_agent(
        model_name="gpt-4.1",
        enable_offloading=cfg["offloading"],
        enable_reduction=cfg["reduction"],
        enable_caching=cfg["caching"],
    )
    print(f"\n[{cfg['name']}]")
    print(f"  Offloading: {'✅' if cfg['offloading'] else '❌'}")
    print(f"  Reduction:  {'✅' if cfg['reduction'] else '❌'}")
    print(f"  Caching:    {'✅' if cfg['caching'] else '❌'}")
    print(f"  에이전트 타입: {type(agent).__name__}")

print("\n" + "=" * 60)
print("모든 에이전트가 성공적으로 생성되었습니다.")

### 권장 설정

| 사용 사례 | Offloading | Reduction | Caching | 이유 |
|----------|------------|-----------|---------|------|
| **짧은 대화** | ❌ | ❌ | ✅ | 오버헤드 최소화 |
| **일반 작업** | ✅ | ❌ | ✅ | 대용량 결과 대비 |
| **장시간 연구** | ✅ | ✅ | ✅ | 모든 최적화 활용 |
| **디버깅** | ❌ | ❌ | ❌ | 전체 컨텍스트 확인 |

---

## 요약

### Context Engineering 5가지 전략 요약

| 전략 | 트리거 조건 | 효과 |
|------|------------|------|
| **Offloading** | 20k 토큰 초과 | 파일로 축출 |
| **Reduction** | 85% 사용량 초과 | Compaction/Summarization |
| **Retrieval** | 파일 접근 필요 | grep/glob 검색 |
| **Isolation** | 복잡한 작업 | SubAgent 위임 |
| **Caching** | 1k+ 토큰 시스템 프롬프트 | Prompt Caching |

### 핵심 인사이트

1. **파일시스템 = 외부 메모리**: 컨텍스트 윈도우는 제한되어 있지만, 파일시스템은 무한
2. **점진적 공개**: 모든 정보를 한 번에 로드하지 않고 필요할 때만 로드
3. **격리된 실행**: SubAgent로 컨텍스트 오염 방지
4. **자동화된 관리**: 에이전트가 직접 컨텍스트를 관리하도록 미들웨어 설계